# Stage I - Prep env and run Primer3 by passing user-defined parameters.
Environment set up  
--- Ubuntu 18.0.4 LTS 
--- Bioconda + Python 3.8.3 
--- Primer3-py 0.6.1

In [1]:
# Load necessary libraries

import primer3
import re
import pandas as pd
import textwrap
import sys

In [2]:
# Open the FASTA file and read the first 500

import re

class InputOutput:
    def __init__(self, file_loc_name = '', data = ''):
        self.file_loc_name = file_loc_name
        self.data = data
        
    def save_as_file(self, file_loc_name = '', data = ''):
        with open (file_loc_name, 'w') as f:
            f.write(data)
            print('Saved!')
            
    def read_seq(self, file_loc_name = '', data = ''):
        with open(file_loc_name, 'r') as f:
            ls = ''
            lines = f.readlines()[1:]
            for line in lines:
                ls  = ls + line.rstrip()
        re.sub('[^a-zA-Z]+', '', ls)
        source_seq = ls[:500]
        print(f'Source sequence is: \n' + source_seq)
        return source_seq
    
new_io = InputOutput()

try:
    read_sequence = new_io.read_seq('/home/qiime2/Downloads/sequence.fasta')
except IOError as e:
    print('An IOError has occurred. Please locate your sequence file correctly.')

Source sequence is: 
GCGCTCTTCCGGCAGCGGTACGTTTGGAGACGCCGGGAACCCGCGTTGGCGTGGTTGACTAGTGCCTCGCAGCCTCAGCATGGGGGAACATGGCCTGGAGCTGGCTTCCATGATCCCCGCCCTGCGGGAGCTGGGCAGTGCCACACCAGAGGAATATAATACAGTTGTACAGAAGCCAAGACAAATTCTGTGTCAATTCATTGACCGGATACTTACAGATGTAAATGTTGTTGCTGTAGAACTTGTAAAGAAAACTGACTCTCAGCCAACCTCCGTGATGTTGCTTGATTTCATCCAGCATATCATGAAATCCTCCCCACTTATGTTTGTAAATGTGAGTGGAAGCCATGAGCGCAAAGGCAGTTGTATTGAATTCAGTAATTGGATCATAACGAGACTTCTGCGGATTGCAGCAACTCCCTCCTGTCATTTGTTACACAAGAAAATCTGTGAAGTCATCTGTTCATTATTATTTCTTTTTAAAAGCAAGAGTCCTGCTA


In [3]:
''' Use Primer3-py, set parameters and run the assay.
Customer specified Ta set as 60C. Since there seems to have no Ta calculation within Primer3-py, 
I use Tm = 65C as a beginning estimate for Ta = 60C (practically, Ta is ~5C lower than Tm).
Later in the result trimming and sorting stage, I will use Ta=0.3xTm_primer+0.7xTm_amplicon-14.9 
to calculate Ta. If other Ta calculation equations are preferred, we can adjust accordingly.
I set MAX and MIN TM parameters for larger range.
'''

import primer3

class SetParamRunPrimer3:
    
    def set_primer_product_size_range(self, lst = [[]]):
        return lst
    
    def set_primer_opt_size(self, num = 20):
        return num
    
    def set_primer_min_size(self, num = 10):
        return num
    
    def set_primer_max_size(self, num = 50):
        return num
    
    def set_primer_internal_opt_size(self, num = 20):
        return num
    
    def set_primer_internal_min_size(self, num = 10):
        return num
    
    def set_primer_internal_max_size(self, num = 50):
        return num
    
    def set_primer_dna_conc(self, conc = 100.0):
        return conc
    
    def set_primer_salt_momovalent(self, conc = 0.0):
        return conc
    
    def set_primer_salt_divalent(self, conc = 0.0):
        return conc
    
    def set_primer_dntp_conc(self, conc = 0.0):
        return conc
    
    def set_primer_dntp_conc(self, conc = 0.0):
        return conc
    
    def set_primer_opt_tm(self, flt = 55.0):
        return flt
    
    def set_primer_internal_opt_tm(self, flt = 55.0):
        return flt
           
new_setrun = SetParamRunPrimer3()

# User specified amplicon size = 100bp
user_specified_amplicon_length = 100

primer_product_size_range = new_setrun.set_primer_product_size_range([[100,100]])
primer_opt_size = new_setrun.set_primer_opt_size(25)
primer_min_size = new_setrun.set_primer_min_size(17)
primer_max_size = new_setrun.set_primer_max_size(35)
primer_internal_opt_size = new_setrun.set_primer_internal_opt_size(25)
primer_internal_min_size = new_setrun.set_primer_internal_min_size(17)
primer_internal_max_size = new_setrun.set_primer_internal_max_size(35)
primer_dna_conc = new_setrun.set_primer_dna_conc(200)
primer_salt_momovalent = new_setrun.set_primer_salt_momovalent(50)
primer_salt_divalent = new_setrun.set_primer_salt_divalent(4.7)
primer_dntp_conc = new_setrun.set_primer_dntp_conc(0.95)

# User-specified Ta = 60C
user_specified_Ta = 60.0

# Optimal Tm is temporarily set as user_specified_Ta +5C, which is 65C
primer_opt_tm = new_setrun.set_primer_opt_tm(user_specified_Ta + 5)
primer_internal_opt_tm = new_setrun.set_primer_internal_opt_tm(user_specified_Ta + 5)

primer_prelim = (primer3.bindings.designPrimers(
    {
        'SEQUENCE_ID': 'ASSAY000001',
        'SEQUENCE_TEMPLATE': read_sequence,
        'SEQUENCE_INCLUDED_REGION': [0,len(read_sequence)]
    },
    {
        'PRIMER_TASK': 'pick_pcr_primers_and_hyb_probe',
        'PRIMER_PICK_LEFT_PRIMER': 1,
        'PRIMER_PICK_INTERNAL_OLIGO': 1,
        'PRIMER_PICK_RIGHT_PRIMER': 1,
        'PRIMER_NUM_RETURN': 60,
        'PRIMER_OPT_SIZE': primer_opt_size,
        'PRIMER_MIN_SIZE': primer_min_size,
        'PRIMER_MAX_SIZE': primer_max_size,
        'PRIMER_INTERNAL_OPT_SIZE': primer_internal_opt_size,
        'PRIMER_INTERNAL_MIN_SIZE': primer_internal_min_size,
        'PRIMER_INTERNAL_MAX_SIZE': primer_internal_max_size,
        'PRIMER_OPT_TM': primer_opt_tm,
        'PRIMER_MAX_TM': 72.0,
        'PRIMER_MIN_TM': 50.0,
        'PRIMER_INTERNAL_OPT_TM': primer_internal_opt_tm,
        'PRIMER_INTERNAL_MAX_TM': 72.0,
        'PRIMER_INTERNAL_MIN_TM': 50.0,
        'PRIMER_PRODUCT_SIZE_RANGE': primer_product_size_range,
        'PRIMER_SALT_MONOVALENT': primer_salt_momovalent,
        'PRIMER_SALT_DIVALENT': primer_salt_divalent,
        'PRIMER_DNTP_CONC': primer_dntp_conc,
        'PRIMER_DNA_CONC': primer_dna_conc,
    }))
print(primer_prelim)

{'PRIMER_LEFT_EXPLAIN': 'considered 7619, GC content failed 44, low tm 133, high tm 2434, high hairpin stability 76, ok 4932', 'PRIMER_RIGHT_EXPLAIN': 'considered 7619, GC content failed 203, low tm 182, high tm 1527, high hairpin stability 16, ok 5691', 'PRIMER_INTERNAL_EXPLAIN': 'considered 9025, GC content failed 205, low tm 1850, high tm 330, high hairpin stability 117, ok 6523', 'PRIMER_PAIR_EXPLAIN': 'considered 96730, unacceptable product size 96667, ok 63', 'PRIMER_LEFT_NUM_RETURNED': 60, 'PRIMER_RIGHT_NUM_RETURNED': 60, 'PRIMER_INTERNAL_NUM_RETURNED': 60, 'PRIMER_PAIR_NUM_RETURNED': 60, 'PRIMER_PAIR_0_PENALTY': 0.6216134498441761, 'PRIMER_LEFT_0_PENALTY': 0.0009057941670675973, 'PRIMER_RIGHT_0_PENALTY': 0.6207076556771085, 'PRIMER_INTERNAL_0_PENALTY': 4.43592378479218, 'PRIMER_LEFT_0_SEQUENCE': 'TGAAATCCTCCCCACTTATGTTTGT', 'PRIMER_RIGHT_0_SEQUENCE': 'CGCAGAAGTCTCGTTATGATCCAAT', 'PRIMER_INTERNAL_0_SEQUENCE': 'GCCATGAGCGCAAAGGCAGTTGTAT', 'PRIMER_LEFT_0': (305, 25), 'PRIMER_RIGHT

# Stage II -  Pre-process results.
--- Read from the original result and extract out info based on primer pair number.
--- Further convert it into Pandas dataframe.

In [4]:
# Put the result into a more structured and manipulable dictionary.

import pandas as pd

class PreProcess():
    def __init__(self, dict):
        self.dict = dict
        
    def pre_process(self, dict):
        initial_dict = {}
        for id in range(dict['PRIMER_PAIR_NUM_RETURNED']):
            primer_id = str(id)
            for key in dict:
                if primer_id in key and not re.search(r'\d', re.sub(r'\D', '', key.replace(primer_id, '', 1))):
                    param = re.sub(r'_([\d]+)', '', key)
                    try:
                        initial_dict[param]
                    except:
                        initial_dict[param] = []
                    finally:
                        initial_dict[param].append(dict[key])
    
        # Convert to Pandas dataframe to facilitate following steps.
        primer = pd.DataFrame.from_dict(initial_dict,orient="index").T
        return primer
    
new_preprocess = PreProcess(primer_prelim)
df = new_preprocess.pre_process(primer_prelim)

In [5]:
df

,PRIMER_PAIR_PENALTY,PRIMER_LEFT_PENALTY,PRIMER_RIGHT_PENALTY,PRIMER_INTERNAL_PENALTY,PRIMER_LEFT_SEQUENCE,PRIMER_RIGHT_SEQUENCE,PRIMER_INTERNAL_SEQUENCE,PRIMER_LEFT,PRIMER_RIGHT,PRIMER_INTERNAL,...,PRIMER_RIGHT_SELF_END_TH,PRIMER_INTERNAL_SELF_END_TH,PRIMER_LEFT_HAIRPIN_TH,PRIMER_RIGHT_HAIRPIN_TH,PRIMER_INTERNAL_HAIRPIN_TH,PRIMER_LEFT_END_STABILITY,PRIMER_RIGHT_END_STABILITY,PRIMER_PAIR_COMPL_ANY_TH,PRIMER_PAIR_COMPL_END_TH,PRIMER_PAIR_PRODUCT_SIZE
0,0.621613,0.000905794,0.620708,4.43592,TGAAATCCTCCCCACTTATGTTTGT,CGCAGAAGTCTCGTTATGATCCAAT,GCCATGAGCGCAAAGGCAGTTGTAT,"(305, 25)","(404, 25)","(344, 25)",...,0,0,0,0,40.2715,2.83,3.16,0,0,100
1,1.38569,0.000905794,1.38478,4.43592,TGAAATCCTCCCCACTTATGTTTGT,CGCAGAAGTCTCGTTATGATCCAA,GCCATGAGCGCAAAGGCAGTTGTAT,"(305, 25)","(404, 24)","(344, 25)",...,0,0,0,0,40.2715,2.83,3.53,0,0,100
2,1.66932,1.04862,0.620708,4.43592,TCCTCCCCACTTATGTTTGTAAATGT,CAATCCGCAGAAGTCTCGTTATGAT,GCCATGAGCGCAAAGGCAGTTGTAT,"(310, 26)","(409, 25)","(344, 25)",...,0,0,0,0,40.2715,2.71,2.45,0,0,100
3,1.89858,1.27787,0.620708,4.43592,TGAAATCCTCCCCACTTATGTTTGTA,CGCAGAAGTCTCGTTATGATCCAAT,GCCATGAGCGCAAAGGCAGTTGTAT,"(305, 26)","(404, 25)","(344, 25)",...,0,0,0,0,40.2715,2.41,3.16,0,0,100
4,1.99063,0.000905794,1.98973,4.43592,TGAAATCCTCCCCACTTATGTTTGT,CGCAGAAGTCTCGTTATGATCCAATT,GCCATGAGCGCAAAGGCAGTTGTAT,"(305, 25)","(404, 26)","(344, 25)",...,0,0,0,0,40.2715,2.83,2.32,0,0,100
5,2.04085,0.000905794,2.03995,4.43592,TGAAATCCTCCCCACTTATGTTTGT,CGCAGAAGTCTCGTTATGATCCA,GCCATGAGCGCAAAGGCAGTTGTAT,"(305, 25)","(404, 23)","(344, 25)",...,0,0,0,0,40.2715,2.83,3.41,0,0,100
6,2.09738,1.47667,0.620708,4.43592,TCCTCCCCACTTATGTTTGTAAATG,CAATCCGCAGAAGTCTCGTTATGAT,GCCATGAGCGCAAAGGCAGTTGTAT,"(310, 25)","(409, 25)","(344, 25)",...,0,0,0,0,40.2715,2.32,2.45,0,0,100
7,2.3081,1.1728,1.13531,4.43592,ATGAAATCCTCCCCACTTATGTTTG,GCAGAAGTCTCGTTATGATCCAATT,GCCATGAGCGCAAAGGCAGTTGTAT,"(304, 25)","(403, 25)","(344, 25)",...,0,0,0,0,40.2715,2.93,2.32,0,0,100
8,2.38487,1.24956,1.13531,4.43592,ATGAAATCCTCCCCACTTATGTTTGT,GCAGAAGTCTCGTTATGATCCAATT,GCCATGAGCGCAAAGGCAGTTGTAT,"(304, 26)","(403, 25)","(344, 25)",...,0,0,0,0,40.2715,2.83,2.32,0,0,100
9,2.4334,1.04862,1.38478,4.43592,TCCTCCCCACTTATGTTTGTAAATGT,CAATCCGCAGAAGTCTCGTTATGA,GCCATGAGCGCAAAGGCAGTTGTAT,"(310, 26)","(409, 24)","(344, 25)",...,0,0,0,0,40.2715,2.71,2.15,0,0,100


# Stage III - Trim results by filtering (quality control).

In [6]:
''' Apply 4 methods to remove unwanted oligos by applying regular expression rules and Primer3 APIs.
* filter_RE: to discard oligos containing restriction enzyme recognition sites by using regular expression rules.
The recognition sites for those user-defined enzymes are CATG|GTAC|TCNNGA. 
Could write a module to crawl the NEB site for corresponding enzymes and there recognition site.
* filter_GC-clamps: to filter out oligos with >=4 G/Cs (out of 5) @3' end by using regular expression rules 
(adjustable, personally I think <=3 G/Cs improves specific binding while too many gives rise to too strong binding).
* filter_multi_cons: to remove oligos having >3 consecutive repeated bases by using regular expression rules.
* filter_hetero_dimer: to throw away heterodimer-forming oligos with Tm 50-75C.
'''

import re

class QualityControl:

    def filter_RE(self, seq):
        return re.search(r'CATG|GTAC|TC\w{2}GA', seq)

    def filter_GC_clamps(self, seq):
        return re.search(r'[GC]{4}', seq[-5:])    

    def filter_multi_cons(self, seq):
        return re.search(r'(.)\1\1\1', seq)

    def filter_hetero_dimer(self, seq1, seq2):
        return 50 < primer3.calcHeterodimer(seq1, seq2, mv_conc=primer_salt_momovalent, 
                                                       dv_conc=primer_salt_divalent, dntp_conc=primer_dntp_conc, 
                                                       dna_conc=primer_dna_conc, temp_c=37, max_loop=30).tm < 75

    def filtering(self, dataframe):
        
        # Drop the columns with non-critical info to reduce data size and expedite processing.
        dataframe.drop(dataframe.columns[[0,1,2,3,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29]], axis = 1, inplace = True)

        # Mark the rows containing filtered targets.
        marked_idx = []
        for idx in range(df.shape[0]):
            primer_left, primer_right, probe = df.at[idx, 'PRIMER_LEFT_SEQUENCE'], df.at[idx, 'PRIMER_RIGHT_SEQUENCE'], df.at[idx, 'PRIMER_INTERNAL_SEQUENCE']

            if self.filter_hetero_dimer(primer_left, primer_right) or self.filter_hetero_dimer(probe, primer_right):
                marked_idx.append(idx)
                break  

            for seq in (primer_left, primer_right, probe): 
                if self.filter_RE(seq) or self.filter_GC_clamps(seq) or self.filter_multi_cons(seq):
                    marked_idx.append(idx)
                    break   

        # Delete those rows containing filtered targets.           
        for row in marked_idx:
            df.drop(index = row, inplace=True)
            
        # Reset index.
        df.reset_index(drop=True, inplace=True)    
        return df
    
new_QC = QualityControl()
df = new_QC.filtering(df)
df

,PRIMER_LEFT_SEQUENCE,PRIMER_RIGHT_SEQUENCE,PRIMER_INTERNAL_SEQUENCE,PRIMER_LEFT,PRIMER_RIGHT,PRIMER_INTERNAL
0,CAGTGCCACACCAGAGGAATATAAT,AGCAACAACATTTACATCTGTAAGT,ACAGAAGCCAAGACAAATTCTGTGT,"(135, 25)","(234, 25)","(168, 25)"
1,CGTGATGTTGCTTGATTTCATCCAG,TTCAATACAACTGCCTTTGCGCTC,TGTTTGTAAATGTGAGTGGAAGCCA,"(273, 25)","(372, 24)","(323, 25)"
2,GCCACACCAGAGGAATATAATACAGT,CTACAGCAACAACATTTACATCTGT,ACAGAAGCCAAGACAAATTCTGTGT,"(139, 26)","(238, 25)","(168, 25)"
3,GTGCCACACCAGAGGAATATAATACA,ACAGCAACAACATTTACATCTGTAAGT,ACAGAAGCCAAGACAAATTCTGTGT,"(137, 26)","(236, 27)","(168, 25)"
4,CGTGATGTTGCTTGATTTCATCCAG,TTCAATACAACTGCCTTTGCGCTCA,TGTTTGTAAATGTGAGTGGAAGCCA,"(273, 25)","(372, 25)","(323, 25)"


# Stage IV - (generate new columns and) Rank results.

In [7]:
'''Generate the following extra columns for sorting and rank:
1. Tm for each oligo.
2. HomodimerTm for each oligo.
3. Temprature differences between predicted Ta and user-designated Ta (60C) for each oligo
(Ta=0.3xTm_primer+0.7xTm_amplicon-14.9).
4. 100bp Amplicon sequence for each predicted PCR product.
5. Oligo length difference between optimal size (25bp) and predicted size for each oligo
(will be used for sorting of the results).
'''

class GenNewCol:

    def __init__(self):
        self.lst_Tm_amplicon = []
        
    # Generate the following lists first so that they can readily become columns of the target dataframe.
    
    def gen_lst_amplicon(self, dataframe, idx_col):
        lst_amplicon = []
        for row in range(dataframe.shape[0]):
            lst_amplicon.append(read_sequence[dataframe.iat[row,idx_col][0]:dataframe.iat[row,idx_col][0]+user_specified_amplicon_length])
            self.lst_Tm_amplicon.append(primer3.calcTm(lst_amplicon[row], mv_conc=primer_salt_momovalent, 
                                                       dv_conc=primer_salt_divalent, dntp_conc=primer_dntp_conc, 
                                                       dna_conc=primer_dna_conc, max_nn_length=60, tm_method='santalucia', salt_corrections_method='santalucia'))
        return lst_amplicon

    def gen_lst_oligo_len_diff(self, dataframe, idx_col):
        lst_oligo_len_diff = []
        for row in range(dataframe.shape[0]):
            lst_oligo_len_diff.append(abs(dataframe.iat[row,idx_col][1]- primer_opt_size)) 
        return lst_oligo_len_diff
    
    def gen_lst_others(self, dataframe, idx_col):
        lst_Tm_oligo, lst_Tm_Homo_oligo, lst_Ta, lst_Ta_diff= [], [], [], []
        for row in range(dataframe.shape[0]):
            lst_Tm_oligo.append(primer3.calcTm(dataframe.iat[row,idx_col], mv_conc=primer_salt_momovalent, 
                                                       dv_conc=primer_salt_divalent, dntp_conc=primer_dntp_conc, 
                                                       dna_conc=primer_dna_conc, max_nn_length=60, tm_method='santalucia', salt_corrections_method='santalucia'))
            lst_Tm_Homo_oligo.append(primer3.calcHomodimer(dataframe.iat[row,idx_col], mv_conc=primer_salt_momovalent, 
                                                       dv_conc=primer_salt_divalent, dntp_conc=primer_dntp_conc, 
                                                       dna_conc=primer_dna_conc, temp_c=37, max_loop=30).tm)
            lst_Ta.append(0.3*lst_Tm_oligo[row] + 0.7*self.lst_Tm_amplicon[row] - 14.9)
            lst_Ta_diff.append(abs(lst_Ta[row] - user_specified_Ta))
        return [lst_Tm_oligo, lst_Tm_Homo_oligo, lst_Ta_diff]
    

    # This is the method to generate new columns.
    def gen_new_col(self, dataframe, col_name, lst):
        dataframe[col_name] = pd.Series(lst)
        return dataframe
    
    # Swap the values of the tuples (oligo_start, oligo_len) for each oligo so that they oligo length can be easily seen.
    def tuple_swap(self, dataframe, idx_col):
        for row in range(dataframe.shape[0]):
            lst_tuple_swap = []
            lst_tuple_swap.append(dataframe.iat[row,idx_col][1])
            lst_tuple_swap.append(dataframe.iat[row,idx_col][0])
            dataframe.iat[row,idx_col] = tuple(lst_tuple_swap)
        return dataframe
    
    # Sort and re-index. Sorting is based on: the differences between the predicted values of oligo length/Ta 
    # and the expected values (25bp and Ta=60, respectively) 
    def rank(self, dataframe):
        df = dataframe.sort_values(by=['PRIMER_LEFT_LENGTH_DIFF','PRIMER_RIGHT_LENGTH_DIFF','PROBE_LENGTH_DIFF','PRIMER_LEFT_Ta_DIFF','PRIMER_RIGHT_Ta_DIFF','PROBE_Ta_DIFF'], ascending=[True, True, True, True, True, True], ignore_index=True).reset_index(drop=True)
        return df
    
gnc = GenNewCol()

# Generate column 'AMPLICON'
gnc.gen_new_col(df, 'AMPLICON', gnc.gen_lst_amplicon(df, 3))

# Generate the following columns   
gnc.gen_new_col(df, 'PRIMER_LEFT_LENGTH_DIFF', gnc.gen_lst_oligo_len_diff(df, 3))
gnc.gen_new_col(df, 'PRIMER_RIGHT_LENGTH_DIFF', gnc.gen_lst_oligo_len_diff(df, 4))
gnc.gen_new_col(df, 'PROBE_LENGTH_DIFF', gnc.gen_lst_oligo_len_diff(df, 5))

lst_lst_LEFT = gnc.gen_lst_others(df, 0)
gnc.gen_new_col(df, 'PRIMER_LEFT_Tm', lst_lst_LEFT[0])
gnc.gen_new_col(df, 'PRIMER_LEFT_Tm_Homo', lst_lst_LEFT[1])
gnc.gen_new_col(df, 'PRIMER_LEFT_Ta_DIFF', lst_lst_LEFT[2])

lst_lst_RIGHT = gnc.gen_lst_others(df, 1)
gnc.gen_new_col(df, 'PRIMER_RIGHT_Tm', lst_lst_RIGHT[0])
gnc.gen_new_col(df, 'PRIMER_RIGHT_Tm_Homo', lst_lst_RIGHT[1])
gnc.gen_new_col(df, 'PRIMER_RIGHT_Ta_DIFF', lst_lst_RIGHT[2])

lst_lst_PROBE = gnc.gen_lst_others(df, 2)
gnc.gen_new_col(df, 'PROBE_Tm', lst_lst_PROBE[0])
gnc.gen_new_col(df, 'PROBE_Tm_Homo', lst_lst_PROBE[1])
gnc.gen_new_col(df, 'PROBE_Ta_DIFF', lst_lst_PROBE[2])
    
# Swap values of (oligo_start, oligo_len) tuples so that sizes of oligos are more visible.
gnc.tuple_swap(df, 3)
gnc.tuple_swap(df, 4)
gnc.tuple_swap(df, 5)

result = gnc.rank(df)
result

,PRIMER_LEFT_SEQUENCE,PRIMER_RIGHT_SEQUENCE,PRIMER_INTERNAL_SEQUENCE,PRIMER_LEFT,PRIMER_RIGHT,PRIMER_INTERNAL,AMPLICON,PRIMER_LEFT_LENGTH_DIFF,PRIMER_RIGHT_LENGTH_DIFF,PROBE_LENGTH_DIFF,PRIMER_LEFT_Tm,PRIMER_LEFT_Tm_Homo,PRIMER_LEFT_Ta_DIFF,PRIMER_RIGHT_Tm,PRIMER_RIGHT_Tm_Homo,PRIMER_RIGHT_Ta_DIFF,PROBE_Tm,PROBE_Tm_Homo,PROBE_Ta_DIFF
0,CAGTGCCACACCAGAGGAATATAAT,AGCAACAACATTTACATCTGTAAGT,ACAGAAGCCAAGACAAATTCTGTGT,"(25, 135)","(25, 234)","(25, 168)",CAGTGCCACACCAGAGGAATATAATACAGTTGTACAGAAGCCAAGA...,0,0,0,65.167290,0.175596,2.311859,61.908076,8.141399,1.334095,65.855789,20.738345,2.518409
1,CGTGATGTTGCTTGATTTCATCCAG,TTCAATACAACTGCCTTTGCGCTCA,TGTTTGTAAATGTGAGTGGAAGCCA,"(25, 273)","(25, 372)","(25, 323)",CGTGATGTTGCTTGATTTCATCCAGCATATCATGAAATCCTCCCCA...,0,0,0,65.512324,-10.904102,3.276370,68.372424,15.184544,4.134400,65.858799,-45.678193,3.380312
2,CGTGATGTTGCTTGATTTCATCCAG,TTCAATACAACTGCCTTTGCGCTC,TGTTTGTAAATGTGAGTGGAAGCCA,"(25, 273)","(24, 372)","(25, 323)",CGTGATGTTGCTTGATTTCATCCAGCATATCATGAAATCCTCCCCA...,0,1,0,65.512324,-10.904102,3.276370,67.039186,15.184544,3.734428,65.858799,-45.678193,3.380312
3,GCCACACCAGAGGAATATAATACAGT,CTACAGCAACAACATTTACATCTGT,ACAGAAGCCAAGACAAATTCTGTGT,"(26, 139)","(25, 238)","(25, 168)",GCCACACCAGAGGAATATAATACAGTTGTACAGAAGCCAAGACAAA...,1,0,0,64.890270,-31.966833,2.228753,62.467270,-17.150366,1.501853,65.855789,20.738345,2.518409
4,GTGCCACACCAGAGGAATATAATACA,ACAGCAACAACATTTACATCTGTAAGT,ACAGAAGCCAAGACAAATTCTGTGT,"(26, 137)","(27, 236)","(25, 168)",GTGCCACACCAGAGGAATATAATACAGTTGTACAGAAGCCAAGACA...,1,2,0,65.103841,-12.992255,2.292825,64.321941,8.141399,2.058255,65.855789,20.738345,2.518409


# Stage V - Report results.
--- Output assay result.

In [8]:
''' Print required contents including oligo sequences, corresponding Tm and Tm_homodimers, and amplicon sequences.
Alternatively, could use sys.stdout to log the report.
'''

# import sys

# class Logger:
#     def __init__(self, filename='default.log'):
#         self.terminal = sys.stdout
#         self.log = open(filename, 'a')

#     def write(self, msg):
#         self.terminal.write(msg)
#         self.log.write(msg)

#     def flush(self):
#         pass

# sys.stdout = Logger('Assay design report.txt')

import textwrap

class Report:
    def report(self, dataframe):
        row_num = dataframe.shape[0]
        print(f'We found ' + str(row_num) + ' groups of oligos. \n')
        for row in range(row_num):
            print(f'# ' + str(row+1))
            print(f'Forward - ' + dataframe.iat[row, 0] + 
                  '  Tm=' + str(round(dataframe.iat[row, 10], 1)) + 
                  '  Tm_homodimer=' + str(round(dataframe.iat[row, 11], 1)))
            print(f'Reverse - ' + dataframe.iat[row, 1] + 
                  '  Tm=' + str(round(dataframe.iat[row, 13], 1)) + 
                  '  Tm_homodimer=' + str(round(dataframe.iat[row, 14], 1)))
            print(f'Probe   - ' + dataframe.iat[row, 2] + 
                  '  Tm=' + str(round(dataframe.iat[row, 16], 1)) + 
                  '  Tm_homodimer=' + str(round(dataframe.iat[row, 17], 1)))
            print(f'Amplicon sequence (100bp) - \n' + textwrap.fill(dataframe.iat[row, 6], width=50) + '\n')
            
result_report = Report()
result_report.report(result)

We found 5 groups of oligos. 

# 1
Forward - CAGTGCCACACCAGAGGAATATAAT  Tm=65.2  Tm_homodimer=0.2
Reverse - AGCAACAACATTTACATCTGTAAGT  Tm=61.9  Tm_homodimer=8.1
Probe   - ACAGAAGCCAAGACAAATTCTGTGT  Tm=65.9  Tm_homodimer=20.7
Amplicon sequence (100bp) - 
CAGTGCCACACCAGAGGAATATAATACAGTTGTACAGAAGCCAAGACAAA
TTCTGTGTCAATTCATTGACCGGATACTTACAGATGTAAATGTTGTTGCT

# 2
Forward - CGTGATGTTGCTTGATTTCATCCAG  Tm=65.5  Tm_homodimer=-10.9
Reverse - TTCAATACAACTGCCTTTGCGCTCA  Tm=68.4  Tm_homodimer=15.2
Probe   - TGTTTGTAAATGTGAGTGGAAGCCA  Tm=65.9  Tm_homodimer=-45.7
Amplicon sequence (100bp) - 
CGTGATGTTGCTTGATTTCATCCAGCATATCATGAAATCCTCCCCACTTA
TGTTTGTAAATGTGAGTGGAAGCCATGAGCGCAAAGGCAGTTGTATTGAA

# 3
Forward - CGTGATGTTGCTTGATTTCATCCAG  Tm=65.5  Tm_homodimer=-10.9
Reverse - TTCAATACAACTGCCTTTGCGCTC  Tm=67.0  Tm_homodimer=15.2
Probe   - TGTTTGTAAATGTGAGTGGAAGCCA  Tm=65.9  Tm_homodimer=-45.7
Amplicon sequence (100bp) - 
CGTGATGTTGCTTGATTTCATCCAGCATATCATGAAATCCTCCCCACTTA
TGTTTGTAAATGTGAGTGGAAGCCATGAGCGCAAAG